In [1]:
import nrrd
from matplotlib import pyplot as plt
import numpy as np
import SimpleITK as sitk
from PIL import Image
import glob
import pydicom
import os
import cv2

from fastai.vision.all import *
from fastai.medical.imaging import*



In [2]:
folders = glob.glob('test/**/*')

In [3]:
len(folders)

98

In [4]:
def file_plane(IOP):
    IOP_round = [round(x) for x in IOP]
    plane = np.cross(IOP_round[0:3], IOP_round[3:6])
    plane = [abs(x) for x in plane]
    if plane[0] == 1:
        return "Sagittal"
    elif plane[1] == 1:
        return "Coronal"
    elif plane[2] == 1:
        return "Transverse"

In [ ]:
for folder in folders:
    se_arr = sitk.ReadImage(folder)
    se_arr= sitk.GetArrayFromImage(se_arr)
    fold = folder.split('\\')[1]
    m = glob.glob(f'prostatex/prostatex/{fold}/**/**/*.dcm')
    os.makedirs(f'prostate-X/images/{fold}/', exist_ok=True)
    os.makedirs(f'prostate-X/masks/{fold}/', exist_ok=True)
    n=[]
    counter=0
    for i,mm in enumerate(m):
        
        f = pydicom.read_file(mm)
        img_arr = sitk.ReadImage(mm)
        img_arr= sitk.GetArrayFromImage(img_arr)[0]
        if img_arr.shape == se_arr[0].shape:
            if file_plane(f.ImageOrientationPatient) =='Transverse':
                if len(n)==len(se_arr):
                    break
                
                img = tensor(img_arr)
                brks = img.freqhist_bins(n_bins=7000);
                data_scaled = img.hist_scaled(brks)
                se_arr[len(se_arr)-counter -1 ] = np.array(se_arr[len(se_arr)-counter -1 ]==2, dtype=int)
                cv2.imwrite(f'prostate-X/images/{fold}/{i:02d}.png', data_scaled.numpy()*255)
                cv2.imwrite(f'prostate-X/masks/{fold}/{i:02d}.png', se_arr[len(se_arr)-counter -1 ])
                counter+=1